# Import af biblioteker  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Indlæsning af data

In [ ]:
df = pd.read_csv('../data/Data_Train.csv')
df.sample(5)

Vi har importeret dataene og lavet en hurtig gennemgang for at få et overblik over datasættet. Nu går vi videre med at forberede og rense dataene, så de er klar til analyse.

# 2. Rensning af data

In [ ]:
# Tjekker for manglende værdier
print("Tjekker for manglende værdier:")
print(df.isnull().sum())

# Tjekker for NaN-værdier
print("Tjekker for NaN-værdier:")
print(df.isna().sum())

# Tjekker for datatyper
print("Tjekker for datatyper:")
print(df.info())

# Tjekker for duplikater
print("Tjekker for duplikater:")
print(df.duplicated().sum()) 

In [ ]:
# Finder alle rækker der er identiske med mindst én anden (keep=False viser ALLE forekomster)
fulde_duplikater = df[df.duplicated(keep=False)]

# Udskriver samlet antal duplikerede rækker
print("Antal fuldstændige duplikatrækker (inkl. kopier):", fulde_duplikater.shape[0])

# Viser et par eksempler på de duplikerede rækker
print("\nEksempler på duplikerede rækker:")
print(fulde_duplikater.head())

# Tæller hvor mange af disse er faktiske kopier (ikke første forekomst)
antal_egentlige_duplikater = df[df.duplicated(keep=False)].duplicated().sum()
print("\nAntal gentagelser (egentlige duplikater):", antal_egentlige_duplikater)

# Tjekker hvor mange unikke rækker er det der gentages?
unikke_rækker = fulde_duplikater.drop_duplicates()
print("Antal unikke rækker blandt de duplikerede:", unikke_rækker.shape[0])

# Eksempel 1: Række 2
print("\nForekomster af række 2:")
print(df[df.eq(df.loc[2]).all(axis=1)])

# Eksempel 2: Række 6
print("\nForekomster af række 6:")
print(df[df.eq(df.loc[6]).all(axis=1)])

In [ ]:
# Fjerner duplikaterne fra datasættet for at sikre, at kun unikke observationer er tilbage
df = df.drop_duplicates()
print("Datasættets nye størrelse efter sletning af duplikater:", df.shape)

Datasættet indeholder ikke nogen manglende værdier, hverken som NaN eller null. Datatyperne består hovedsageligt af tekst (object), som vi vil konvertere til string, samt nogle numeriske kolonner, der er defineret som heltal (int64) og decimaltal (float64).

Datasættet indeholder en del duplikater, hvor nogle rækker er præcist ens. Selvom duplikater kan være nyttige i nogle analyser, har vi valgt at fjerne dem her. Det gør vi for at sikre, at hver række i datasættet bidrager unikt til vores analyser og modeller. Ved at fjerne duplikaterne undgår vi, at nogle observationer får for stor vægt i analysen, hvilket kan påvirke resultaterne. Dette er vigtigt, både for lineær regression og andre statistiske analyser og maskinlæringsmodeller.

Nu opdeles datasættet i to separate dataframes: en med numeriske værdier og en med nominelle (kategoriske) data, da det bliver lettere at arbejde med disse typer.

In [ ]:
# Fjerner 'Dep_Time' kolonnen, da den ikke er relevant for analysen
df.drop(['Dep_Time'], axis=1, inplace=True) 

In [ ]:
# Konverterer kolonnen 'Date_of_Journey' fra tekstformat til datetime-objekter med formatet dag/måned/år
df['Date_of_Journey'] = pd.to_datetime(df['Date_of_Journey'], format='%d/%m/%Y')

# Ekstraherer måneden fra 'Date_of_Journey' og gemmer den som en ny kolonne 'Journey_month' som int64
df['Journey_month'] = df['Date_of_Journey'].dt.month.astype('int64')

# Fjerner den oprindelige 'Date_of_Journey'-kolonne, da vi har ekstraheret måneden og kun denne er relevant fremover
df.drop('Date_of_Journey', axis=1, inplace=True)

De resterende kolonner beholdes i datasættet, da de kan være relevante i senere analyser.

In [ ]:
# Opdaterer 'Class'-kolonnen baseret på information i 'Airline', 
# da den oprindelige 'Class'-kolonne kun indeholdt null-værdier og derfor var ubrugelig.
df['Class'] = df['Airline'].apply(
    lambda x: 'Business' if 'Business' in x 
    else 'Premium economy' if 'Premium economy' in x 
    else 'Standard'
)

# Omdanner 'Class' fra tekst til numerisk værdi i en ny kolonne 'class_numb', 
# så den kan bruges til numerisk analyse som fx korrelationsmatrix.

df['class_numb'] = df['Class'].map({
    'Standard': 0,
    'Premium economy': 1,
    'Business': 2
})

# Tjekker mappingen af 'Class' til numeriske værdier
class_mapping = {
    'Standard': 0,
    'Premium economy': 1,
    'Business': 2
}

# Viser mappingen
print("Mapping af klasser til numeriske værdier:")
print(class_mapping)

# Viser nogle rækker for at bekræfte korrekt omkodning af 'Class'
print(df[['Class', 'class_numb']].head())

In [ ]:
# Omdanner 'Airline' fra tekst til numerisk værdi i en ny kolonne 'airline_numb', 
# så den kan bruges til numerisk analyse som fx korrelationsmatrix.

# Laver en liste over unikke flyselskaber
unique_airlines = df['Airline'].unique()

# Laver en dictionary til mapping, f.eks. 'airline_numb': 0, 1, 2 osv.
airline_mapping = {airline: i for i, airline in enumerate(unique_airlines)}

# Anvender mapping til at lave en ny numerisk kolonne
df['airline_numb'] = df['Airline'].map(airline_mapping)

# Tjekker mappingen
print("Mapping af flyselskaber til numeriske værdier:")
print(airline_mapping)

# Viser nogle rækker for at bekræfte
print(df[['Airline', 'airline_numb']].head())

In [ ]:
# One-hot encoder både 'Class' og 'Airline' kolonnerne
dfEncoded = pd.get_dummies(df[['Class', 'Airline']], prefix=['Class', 'Airline'], dtype=pd.Int64Dtype())

# Kombinerer 'Price', 'Journey_month' og de one-hot encodede kolonner til et datasæt
dfNumeric = pd.concat([df[['Price', 'Journey_month']].reset_index(drop=True), dfEncoded.reset_index(drop=True)], axis=1)

# Viser et tilfældigt udsnit
dfNumeric.sample(5)

In [ ]:
# Konverterer alle 'object'-kolonner til 'string'-type
for col in df:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('string')

# Tjekker datatyperne efter konvertering
df.dtypes

# 3. Dataudforskning

In [ ]:
df.describe()

Price:
Priserne varierer fra 1.759 til 79.512, med en gennemsnitspris på ca. 8.435. Den høje standardafvigelse (4.605) indikerer stor variation i priserne, sandsynligvis som følge af forskelle i rejseklasse og andre forhold.

Journey_month: 
Rejserne fordeler sig over månederne marts (3) til juni (6), med en gennemsnitlig rejsemåned på ca. 4,7, hvilket svarer til midten af april. Den relativt høje standardafvigelse (ca. 1,16) viser, at rejserne spreder sig over hele perioden, men der er en tendens til flere rejser i april og maj (median på 5). Fordelingen indikerer, at rejsemåneden kan have betydning for prisvariationer og bør inkluderes i analysen.

class_numb:
Næsten alle billetter er i Standard-klassen (0), hvilket ses ved, at både median og alle kvartiler er 0. Den ekstremt lave gennemsnitsværdi (0,003) og standardafvigelse (0,067) viser, at Premium economy (1) og Business (2) kun forekommer i et meget lille antal tilfælde.

Denne stærkt ubalancerede fordeling er problematisk i forbindelse med statistisk analyse og maskinlæringsmodeller f.eks. regressionsanalyse eller klassifikationsalgoritmer. Modellerne får svært ved at lære meningsfulde mønstre for de sjældne klasser, hvilket kan føre til skæve resultater, overfitting på Standard-klassen og lav præcision for de øvrige klasser. For at opnå mere robuste og generaliserbare modeller ville en mere balanceret klasserepræsentation være ønskelig.

airline_numb:
Flyenes selskaber er kodet numerisk fra 0 til 11, hvilket svarer til 12 unikke flyselskaber i datasættet. Den gennemsnitlige værdi på ca. 2,16 viser, at observationerne overvejende er koncentreret omkring de lavere numeriske koder, hvilket kan afspejle, at nogle flyselskaber optræder hyppigere end andre. Standardafvigelsen på 1,90 indikerer, at der er en relativt bred spredning mellem de forskellige flyselskaber, hvilket giver en god variation til at undersøge forskelle i pris og prisvariationer på tværs af selskaber. Denne spredning er vigtig, da den muliggør en meningsfuld analyse af, hvordan prisniveau og variation kan afhænge af flyselskabet, hvilket er centralt i vurderingen af mulige grupperinger eller clusters baseret på flyselskabsdata.

In [ ]:
df.columns

Vi kan se, at dataene allerede er forholdsvis standardiserede, men lad os konvertere kolonnenavnene til at bruge "x-y" i stedet for "x_y" for at demonstrere muligheden for at forberede data.

In [ ]:
import gc

dfStan = df.copy()

# Skifter kolonnenavne til store bogstaver og erstatter "_" med "-"
dfStan.columns = [col.upper().replace('_', '-') for col in dfStan.columns]

print(dfStan.columns)

del dfStan

# Kører garbage collection for at rydde op i hukommelsen
gc.collect()

Som nævnt foretager vi ikke en permanent ændring af kolonnenavnene i datasættet, men viser blot, hvordan det kan gøres. 

Vi vælger dog at konvertere kolonnenavnene til små bogstaver, da det giver et pænere, mere ensartet og læsevenligt datasæt.

In [ ]:
# Skifter kolonnenavne til små bogstaver 
df.columns = [col.lower() for col in df.columns]

print(df.columns)

For at opnå en bedre forståelse af prisdataenes fordeling udfører vi først grundlæggende visualiseringer.

In [ ]:
from scipy.stats import skew, kurtosis  

plt.figure(figsize=(10, 8))

# Histogram med KDE øverst
plt.subplot(2, 1, 1)
sns.histplot(df['price'], kde=True, bins=50)
plt.title('Fordeling af Priser - histogram')
plt.ylabel('Antal')
plt.xlabel('Pris')

# Boxplot nederst
plt.subplot(2, 1, 2)
sns.boxplot(x=df['price'])
plt.title('Boxplot af Pris')

plt.tight_layout()
plt.show()

# Beregning af skævhed og kurtosis
skewness = skew(df['price'].dropna())
kurt = kurtosis(df['price'].dropna())

print(f"Skævhed (Skewness): {skewness}")
print(f"Kurtosis: {kurt}")

if skewness > 0:
    print("Fordelingen er højreskæv (positiv skævhed).")
elif skewness < 0:
    print("Fordelingen er venstreskæv (negativ skævhed).")
else:
    print("Fordelingen er symmetrisk.")

if kurt > 3:
    print("Fordelingen er spidsere end en normalfordeling (høj kurtosis).")
elif kurt < 3:
    print("Fordelingen er fladere end en normalfordeling (lav kurtosis).")
else:
    print("Fordelingen har samme spidshed som en normalfordeling.")


Fordelingen er højreskæv (positiv skævhed), hvilket betyder, at der er flere lavere værdier, og den højre hale af fordelingen er længere.

Fordelingen har også høj kurtosis, hvilket indikerer, at dataene er spidsere end en normalfordeling, og der er flere ekstreme værdier (outliers) end vi ville forvente i en normalfordeling. Dette tyder på, at vores data kan indeholde nogle ekstreme observationer, som vi vil fjerne ved hjælp af z-score og IQR teknikkerne for at forbedre modellens præcision.

In [ ]:
from datarenser import get_no_outliers_df_train

# Fjerner både IQR‑outliers og z-score‑outliers med et metodekald
df_final = get_no_outliers_df_train(df)

# Tjekker dimensioner før/efter
print(f"Før outlier-fjernelse: {df.shape}")
print(f"Efter outlier-fjernelse: {df_final.shape}")

In [ ]:
from scipy.stats import skew, kurtosis  

plt.figure(figsize=(10, 8))

# Histogram efter outlier-fjernelse
plt.subplot(2, 1, 1)
sns.histplot(df_final['price'], kde=True, bins=50)
plt.title('Histogram af Pris Efter outlier-fjernelse')
plt.xlabel('Pris')
plt.ylabel('Antal')

# Boxplot efter outlier-fjernelse
plt.subplot(2, 1, 2)
sns.boxplot(x=df_final['price'])
plt.title('Boxplot af Pris Efter outlier-fjernelse')

plt.tight_layout()
plt.show()

# Beregning af skævhed og kurtosis efter outlier-fjernelse
skewness_after = skew(df_final['price'].dropna())
kurt_after = kurtosis(df_final['price'].dropna())

print(f"Skævhed (Skewness): {skewness_after:.2f}")
print(f"Kurtosis: {kurt_after:.2f}")

if skewness_after > 0:
    print("Fordelingen er højreskæv (positiv skævhed).")
elif skewness_after < 0:
    print("Fordelingen er venstreskæv (negativ skævhed).")
else:
    print("Fordelingen er symmetrisk.")

if kurt_after > 3:
    print("Fordelingen er spidsere end en normalfordeling (høj kurtosis).")
elif kurt_after < 3:
    print("Fordelingen er fladere end en normalfordeling (lav kurtosis).")
else:
    print("Fordelingen har samme spidshed som en normalfordeling.")

In [ ]:
# Scatterplot: price vs class
plt.figure(figsize=(8, 6))
sns.stripplot(data=df, x='class', y='price', jitter=True, alpha=0.6)
plt.title('Scatterplot: Pris pr. klasse (med outliers)')
plt.xlabel('Billetklasse')
plt.ylabel('Pris')
plt.tight_layout()
plt.show()

# Histogram: prisfordeling pr. klasse
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='price', hue='class', bins=50, kde=True, multiple='stack')
plt.title('Histogram: Prisfordeling pr. klasse (med outliers)')
plt.xlabel('Pris')
plt.ylabel('Antal')
plt.tight_layout()
plt.show()

# Boxplot: Pris pr. klasse
plt.figure(figsize=(8, 6))
sns.boxplot(data=df, x='class', y='price')
plt.title('Boxplot: Pris pr. klasse (med outliers)')
plt.xlabel('Billetklasse')
plt.ylabel('Pris')
plt.tight_layout()
plt.show()

# Beregner og udskriver gennemsnitspriser pr. klasse sorteret i stigende rækkefølge med outliers
mean_prices = df.groupby('class')['price'].mean().sort_values()

print("Gennemsnitspriser pr. klasse (med outliers):")
for klasse, pris in mean_prices.items():
    print(f"{klasse}: {pris:.2f}")


In [ ]:
# Scatterplot: price vs class
plt.figure(figsize=(8, 6))
sns.stripplot(data=df_final, x='class', y='price', jitter=True, alpha=0.6)
plt.title('Scatterplot: Pris pr. klasse (uden outliers)')
plt.xlabel('Billetklasse')
plt.ylabel('Pris')
plt.tight_layout()
plt.show()

# Histogram: prisfordeling pr. klasse
plt.figure(figsize=(10, 6))
sns.histplot(data=df_final, x='price', hue='class', bins=50, kde=True, multiple='stack')
plt.title('Histogram: Prisfordeling pr. klasse (uden outliers)')
plt.xlabel('Pris')
plt.ylabel('Antal')
plt.tight_layout()
plt.show()

# Boxplot: Pris pr. klasse
plt.figure(figsize=(8, 6))
sns.boxplot(data=df_final, x='class', y='price')
plt.title('Boxplot: Pris pr. klasse (uden outliers)' )
plt.xlabel('Billetklasse')
plt.ylabel('Pris')
plt.tight_layout()
plt.show()

# Beregner og udskriver gennemsnitspriser pr. klasse sorteret i stigende rækkefølge uden outliers
mean_prices = df_final.groupby('class')['price'].mean().sort_values()

print("Gennemsnitspriser pr. klasse (uden outliers):")
for klasse, pris in mean_prices.items():
    print(f"{klasse}: {pris:.2f}")


Under analysen blev Business-klassen fjernet som outliers på grund af deres naturligt højere priser sammenlignet med de øvrige klasser. Da disse priser er væsentlige for forståelsen af prisstrukturen, har vi valgt ikke at fjerne disse outliers for at bevare vigtig information. Derfor fortsætter vi analysen med datasættet inklusive outliers, så vi bedre kan afspejle, at Business-klassen har en højere pris end standardversionerne inden for samme flyselskab.

Resultatet viser tydeligt, at priserne varierer betydeligt mellem standard og premium-versionerne inden for samme flyselskab.

Gennemsnitsprisen for standardklassen ligger omkring 8.396, mens Premium Economy har en højere gennemsnitspris på cirka 10.938. Dette indikerer, at Premium Economy tilbyder ekstra komfort eller services, som reflekteres i den højere pris.

Business-klassen skiller sig markant ud med en gennemsnitspris på hele 58.359, hvilket er væsentligt højere end både standard og Premium Economy. Dette understreger, at Business er en premium-version med en betydeligt højere pris, som afspejler dens luksus og ekstra ydelser.

Sammenfattende viser analysen, at premium-versionerne, især Business-klassen, har en markant højere pris end standardversionerne, hvilket bekræfter, at prisniveauet stiger i takt med billetklassens premium-status inden for samme flyselskab.

In [ ]:
# Scatterplot: price vs journey_month
plt.figure(figsize=(8, 6))
sns.stripplot(data=df, x='journey_month', y='price', jitter=True, alpha=0.6)
plt.title('Scatterplot: Pris pr. rejsemåned (med outliers)')
plt.xlabel('Rejsemåned')
plt.ylabel('Pris')
plt.tight_layout()
plt.show()

# Histogram: prisfordeling pr. rejsemåned
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='price', hue='journey_month', bins=50, kde=True, multiple='stack')
plt.title('Histogram: Prisfordeling pr. rejsemåned (med outliers)')
plt.xlabel('Pris')
plt.ylabel('Antal')
plt.tight_layout()
plt.show()

# Boxplot: Pris pr. rejsemåned
plt.figure(figsize=(8, 6))
sns.boxplot(data=df, x='journey_month', y='price')
plt.title('Boxplot: Pris pr. rejsemåned (med outliers)')
plt.xlabel('Rejsemåned')
plt.ylabel('Pris')
plt.tight_layout()
plt.show()

mean_prices = df.groupby('journey_month')['price'].mean().sort_index()

print("Gennemsnitspriser pr. rejsemåned (med outliers):")
for month, pris in mean_prices.items():
    print(f"{month}: {pris:.2f}")

In [ ]:
# Scatterplot: price vs journey_month
plt.figure(figsize=(8, 6))
sns.stripplot(data=df_final, x='journey_month', y='price', jitter=True, alpha=0.6)
plt.title('Scatterplot: Pris pr. rejsemåned (uden outliers)')
plt.xlabel('Rejsemåned')
plt.ylabel('Pris')
plt.tight_layout()
plt.show()

# Histogram: prisfordeling pr. rejsemåned
plt.figure(figsize=(10, 6))
sns.histplot(data=df_final, x='price', hue='journey_month', bins=50, kde=True, multiple='stack')
plt.title('Histogram: Prisfordeling pr. rejsemåned (uden outliers)')
plt.xlabel('Pris')
plt.ylabel('Antal')
plt.tight_layout()
plt.show()

# Boxplot: Pris pr. rejsemåned
plt.figure(figsize=(8, 6))
sns.boxplot(data=df_final, x='journey_month', y='price')
plt.title('Boxplot: Pris pr. rejsemåned (uden outliers)')
plt.xlabel('Rejsemåned')
plt.ylabel('Pris')
plt.tight_layout()
plt.show()

# Gennemsnitspriser pr. rejsemåned
mean_prices = df_final.groupby('journey_month')['price'].mean().sort_index()

print("Gennemsnitspriser pr. rejsemåned (uden outliers):")
for month, pris in mean_prices.items():
    print(f"{month}: {pris:.2f}")

Priserne varierer tydeligt afhængigt af rejsemåneden. Den laveste gennemsnitspris findes i april (måned 4) med cirka 5.472, hvilket tyder på, at det er en mere økonomisk måned at rejse i. Priserne stiger markant i maj og juni (måneder 5 og 6) til omkring 8.300-8.230, hvilket indikerer øget efterspørgsel eller sæsonbestemte prisstigninger i disse måneder. Den højeste gennemsnitspris observeres i marts (måned 3) med cirka 9.286 uden outliers og endnu højere (ca. 10.135) når outliers inkluderes. Dette antyder, at rejser i marts generelt er dyrere, muligvis grundet ferieperioder eller andre faktorer, der påvirker efterspørgslen. 

Samlet set viser analysen, at rejsemåneden har en væsentlig indflydelse på prisniveauet især skiller marts sig ud som den dyreste måned, mens april fremstår som den billigste.

In [ ]:
# Lineplot: Prisudvikling pr. flyselskab over rejsemåneder med outliers inkluderet
plt.figure(figsize=(14, 7))
sns.lineplot(data=df, x='journey_month', y='price', hue='airline', marker='o')
plt.title('Lineplot: Gennemsnitspris pr. Flyselskab pr. Måned (med outliers)')
plt.xlabel('Rejsemåned')
plt.ylabel('Pris')
plt.legend(title='Flyselskab', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Lineplot: Prisudvikling pr. flyselskab over rejsemåneder uden outliers
plt.figure(figsize=(14, 7))
sns.lineplot(data=df_final, x='journey_month', y='price', hue='airline', marker='o')
plt.title('Lineplot: Gennemsnitspris pr. Flyselskab pr. Måned (uden outliers)')
plt.xlabel('Rejsemåned')
plt.ylabel('Pris')
plt.legend(title='Flyselskab', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Barplot: Gennemsnitspris pr. klasse og flyselskab
mean_prices_per_class = df.groupby(['airline', 'class'])['price'].mean().reset_index()

plt.figure(figsize=(14, 7))
sns.barplot(data=mean_prices_per_class, x='airline', y='price', hue='class')
plt.title('Barplot: Gennemsnitspris pr. Klasse og Flyselskab')
plt.ylabel('Pris')
plt.xlabel('Flyselskab')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Boxplot: Prisvariation pr. flyselskab med outliers
plt.figure(figsize=(14, 7))
sns.boxplot(data=df, x='airline', y='price')
plt.title('Boxplot: Prisvariation pr. Flyselskab (med outliers)')
plt.xlabel('Flyselskab')
plt.ylabel('Pris')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Boxplot: Prisvariation pr. flyselskab uden outliers
plt.figure(figsize=(14, 7))
sns.boxplot(data=df_final, x='airline', y='price')
plt.title('Boxplot: Prisvariation pr. Flyselskab (uden outliers)')
plt.xlabel('Flyselskab')
plt.ylabel('Pris')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Boxplot: Prisvariation pr. flyselskab og klasse (med outliers)
plt.figure(figsize=(14, 7))
sns.boxplot(data=df, x='airline', y='price', hue='class')
plt.title('Prisvariation pr. Flyselskab og Klasse (Boxplot)')
plt.xlabel('Flyselskab')
plt.ylabel('Pris')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Vælger de relevante kolonner til korrelationsanalyse: pris og den omkodede klasse
cols_to_corr = ['price', 'class_numb']
print("Valgte kolonner til korrelationsanalyse:", cols_to_corr)

# Beregner korrelationsmatrix for valgte variabler
corr = df[cols_to_corr].corr()
print("Beregnet korrelationsmatrix:")
print(corr)

# Visualiserer korrelationsmatrixen som heatmap
plt.figure(figsize=(5,4))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Korrelationsmatrix: Pris vs. Klasse")
plt.show()

Korrelationsmatrixen viser en svag til moderat positiv sammenhæng på omkring 0,24 mellem pris og flyklasse (class_numb). Det indikerer, at prisen typisk øges, jo højere klassen er (Standard < Premium economy < Business), men denne relation er ikke særlig stærk. Det tyder på, at der også er andre faktorer, der spiller ind på prisen.

In [ ]:
avg_prices = df.groupby(['airline', 'class'])['price'].mean().unstack()
print("\nGennemsnitspriser pr. flyselskab og klasse:")
print(avg_prices)

Gennemsnittene bekræfter, at Business class generelt har de højeste priser, efterfulgt af Premium economy, mens Standard class er billigst. Dog er der flyselskaber, som kun tilbyder visse klasser i datasættet, hvilket fremgår af NaN-værdierne. Eksempelvis har 'Jet Airways Business' kun Business klasse-data, mens 'Vistara Premium economy' kun har Premium economy-priser. Standard klassen er tilgængelig på alle flyselskaber og har de laveste gennemsnitspriser.

In [ ]:
# Udvælgelse af kolonner til korrelationsanalyse: pris og rejsemåned
cols_to_corr = ['price', 'journey_month']

# Beregn korrelationsmatrix
corr = df[['price', 'journey_month']].corr()

# Udskriv korrelation
print("Korrelationsmatrix mellem pris og rejsemåned:")
print(corr)

# Visualiser som heatmap
plt.figure(figsize=(5, 4))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Korrelationsmatrix: Pris vs. Rejsemåned")
plt.show()

Der er udvalgt kolonnerne price og journey_month for at undersøge sammenhængen mellem pris og rejsemåned. Korrelationsmatrixen viser en svag negativ korrelation på omkring -0,11, hvilket antyder, at priserne i gennemsnit falder en smule fra marts til juni.

Selvom der er en svag tendens til lavere priser i de senere måneder, er korrelationen så lav, at rejsemåneden alene næsten ingen betydning har for prisvariationerne.

In [ ]:
# Udvælger relevante numeriske kolonner
corr_df = df[['price', 'class_numb', 'journey_month', 'airline_numb']]

# Beregner korrelationer mellem de udvalgte variabler
corr = corr_df.corr()

# Printer korrelationsmatrixen
print("Korrelationsmatrix:")
print(corr)

# Visualiserer korrelationerne med heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Korrelationsmatrix – Pris, Måned, Klasse og Flyselskab')
plt.tight_layout()
plt.show()

Korrelationsmatrixen viser, at der er en svag negativ sammenhæng mellem pris og rejsemåned (-0,11). Dette indikerer, at priserne generelt falder en smule over månederne 3 til 6, men sammenhængen er ikke stærk eller lineær. Faktisk ses fra gennemsnitspriserne, at prisen falder markant i april (måned 4), men derefter stiger igen i maj og juni. Denne uregelmæssige variation gør, at korrelationskoefficienten kun fanger en svag lineær tendens.

Derudover viser matrixen en moderat positiv sammenhæng mellem pris og klasse (0,24), hvilket tyder på, at højere klasser (f.eks. Business) generelt har højere priser. Sammenhængen mellem pris og flyselskab er svag positiv (0,09), hvilket antyder, at priserne varierer lidt afhængigt af flyselskabet, men denne effekt er begrænset.

Kort sagt afspejler korrelationsmatrixen, at prisvariationerne i højere grad er påvirket af klassen end af rejsemåneden eller flyselskabet, og at prisudviklingen over rejsemånederne ikke følger et simpelt lineært mønster.

In [ ]:
mean_std_by_airline = df.groupby('airline')['price'].agg(['mean', 'std']).sort_values(by='mean', ascending=False)
print("Gennemsnitspris og standardafvigelse pr. flyselskab:\n")
print(mean_std_by_airline)

Analysen af gennemsnitspriser og prisvariationer pr. flyselskab viser tydelige forskelle i både prisniveau og prisudsving på tværs af selskaberne. Jet Airways Business har den højeste gennemsnitspris på omkring 58.359, hvilket afspejler, at det er en premium-klasse med markant højere priser end de øvrige flyselskaber. Andre flyselskaber som Multiple carriers Premium economy og Jet Airways befinder sig også i den højere prisklasse, men med væsentligt lavere gennemsnitspriser. I den lave ende finder vi flyselskaber som SpiceJet og Trujet, der har gennemsnitspriser under 5.000. Ved at se på standardafvigelsen, som angiver prisvariation, konstateres det, at Jet Airways Business har en stor prisvariation, hvilket kan skyldes forskellige prisstrategier eller sæsonbestemte udsving. Flyselskaber med lavere standardafvigelse har derimod mere stabile priser. Resultatet af denne analyse giver værdifuld indsigt i, hvordan flyselskaber adskiller sig både med hensyn til prisniveau og prisvariation, og udgør et solidt grundlag for videre segmentering og clustering.

In [ ]:
print("\nObservationer og hypoteser til clustering:")
for airline, row in mean_std_by_airline.iterrows():
    if row['std'] > 10000:
        print(f"• {airline}: Høj prisvariation – potentielt eget cluster")
    elif row['std'] < 3000:
        print(f"• {airline}: Stabil pris – muligvis i gruppe med andre stabile selskaber")

Før vi anvender k-means clustering til at gruppere flyselskaber baseret på prisvariationer, har vi analyseret gennemsnitspriser og standardafvigelser pr. flyselskab for bedre at forstå dataens karakteristika. Resultaterne viser, at Jet Airways Business adskiller sig markant med en høj gennemsnitspris og en stor prisvariation, hvilket indikerer, at dette flyselskab sandsynligvis vil udgøre et selvstændigt cluster.

De øvrige flyselskaber, herunder Multiple carriers Premium economy, Vistara Premium economy, Vistara, GoAir, IndiGo, Air Asia og SpiceJet, viser relativt stabile priser med lavere standardafvigelser. Disse selskaber forventes at gruppere sig i fælles clustre karakteriseret ved lav prisvariation.

Denne indledende analyse bidrager til at forme vores forventninger til clustering-resultatet.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Tjek for manglende værdier før skalering
print("Manglende værdier før skalering:")
print(dfNumeric[['Price', 'Journey_month']].isnull().sum())

# Standard skalering af 'Price'
scaler = StandardScaler()
dfNumeric[['Price', 'Journey_month']] = scaler.fit_transform(dfNumeric[['Price', 'Journey_month']])

# Udskriv et udsnit af data efter skalering
print("\nSkalerede numeriske features + one-hot encoded kategorier (første 5 rækker):")
print(dfNumeric.head())

# Tjek middelværdi og standardafvigelse efter skalering
print("\nMiddelværdi og standardafvigelse efter skalering for 'Price':")
print(dfNumeric[['Price', 'Journey_month']].agg(['mean', 'std']))

Middelværdien af de skalerede features ligger tæt på 0 (alle omkring ±0.00000), og standardafvigelsen er cirka 1.00006 for alle variabler. Dette bekræfter, at standardiseringen er udført korrekt, og at data nu er centreret og skaleret ensartet. Der er ingen manglende værdier i de anvendte features. Data er derfor klar til clustering med k-means, som kan bruges til at kategorisere flyselskaber baseret på prisvariationer over rejsemåneden samt forskelle mellem standard og premium-versioner.

# 4. Datamodellering